In [1]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

'2.0.0'

In [2]:
#建议采用GPU进行训练，需要使用tensorflow-gpu-2.0并设置memory_growth
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu,True)

# 卷积网络简介

   卷积网络接受形状为（image_height，image_width，image_channels）（不包括批量维度）的输入张量。在本例中，我们将配置卷积网络以处理大小为（28，28，1）的输入，这是MNIST图像的格式。我们将参数input_shape =（28，28，1）传递给第一层，来完成设置。

In [3]:
# 5-1实例化一个小型的卷积神经网络
model=tf.keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3),  activation='relu',input_shape=(28,28,1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))  
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

In [4]:
# 查看目前卷积神经网络的框架
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


# 
可以在上面看到，每个Conv2D和MaxPooling2D层的输出形状为（高度，宽度，通道）。 随着网络的加深，宽度和高度尺寸趋于缩小。 通道数由传递给Conv2D层（例如32或64）的第一个参数控制。

下一步是将我们最后的输出张量（形状为（3，3，64））输入到一个密集连接的分类器网络中，就像您已经熟悉的那些：Dense层。 这些分类器可以处理1维向量，而我们当前的输出是3维张量。 首先，我们必须将3维输出展平为1维，然后添加几个Dense层：

In [5]:
# 5-2 在卷积神经网络上添加分类器
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 
我们将进行10类别分类，因此我们使用具有10个输出的softmax激活的最终层。 

In [6]:
# 查看网络结构
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

# 
在经过两个全连接层之前，我们的（3，3，64）输出被展平为形状为（576，）的向量。


In [8]:
# 5-3 在MNIST图像上训练卷积神经网络
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = tf.one_hot(train_labels, depth=10)
test_labels = tf.one_hot(test_labels, depth=10)

In [9]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 135us/sample - loss: 0.1722 - accuracy: 0.9471
Epoch 2/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0464 - accuracy: 0.9856
Epoch 3/5
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0314 - accuracy: 0.9900
Epoch 4/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0243 - accuracy: 0.9930
Epoch 5/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0191 - accuracy: 0.9944


In [10]:
# 在测试数据上对模型进行评估
test_loss,test_acc=model.evaluate(test_images,test_labels)
test_acc

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.9919